In [31]:
import pandas as pd
import math
from sklearn import neighbors, datasets
from numpy.random import permutation
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

cols = ['methodology', 'requirements_volatility', 
        'requirements_clarity', 'dev_time', 'project_size', 'team_size', 
        'prod_complexity', 'testing_intensity', 'risk_analysis', 'user_participation',
        'team_expertise', 'dev_expertise', 'doc_needed', 'fund_avail', 'delivery_speed', 'task_visualisation']
        
num_cols = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

df = pd.read_csv('SDLC2.csv', names = cols, usecols=num_cols, header = 0)

df.head()

,methodology,requirements_volatility,requirements_clarity,dev_time,project_size,team_size,prod_complexity,testing_intensity,risk_analysis,user_participation,team_expertise,dev_expertise,doc_needed,fund_avail,delivery_speed,task_visualisation
0,Scrum,Changing,unknown/defined later in the lifecycle,Intensive,Medium,Medium (6-15),Simple,After each cycle (Intensive testing),Low,High,High,High,Low,High,Medium,Low
1,Kanban,Changing,unknown/defined later in the lifecycle,Intensive,Medium,Large (16....),Complex,After each cycle (Intensive testing),Medium,Low,High,High,Medium,High,Medium,High
2,Scrum,Changing,unknown/defined later in the lifecycle,Intensive,Medium,Medium (6-15),Simple,After each cycle (Intensive testing),Low,High,High,High,Low,High,Medium,Low
3,Hybrid: Scrum and Waterfall,Changing,understandable/early defined,Intensive,Medium,Large (16....),Complex,After each cycle (Intensive testing),Medium,High,Medium,High,High,High,Medium,Low
4,Hybrid: Scrum and Waterfall,Changing,understandable/early defined,Intensive,Medium,Medium (6-15),Complex,After each cycle (Intensive testing),High,Medium,High,High,Medium,High,Medium,Low


In [32]:
def convert_to_vectors(df):

    df['risk_analysis'] = df['risk_analysis'].map(dict(Low=1, Medium=2,High=3))
    df['user_participation'] = df['user_participation'].map(dict(Low=1, Medium=2,High=3))
    df['team_expertise'] = df['team_expertise'].map(dict(Low=1, Medium=2,High=3))
    df['dev_expertise'] = df['dev_expertise'].map(dict(Low=1, Medium=2,High=3))
    df['doc_needed'] = df['doc_needed'].map(dict(Low=1, Medium=2,High=3))
    df['fund_avail'] = df['fund_avail'].map(dict(Low=1, Medium=2,High=3))
    df['delivery_speed'] = df['delivery_speed'].map(dict(Low=1, Medium=2,High=3))
    df['task_visualisation'] = df['task_visualisation'].map(dict(Low=1, Medium=2,High=3))

    # project_type = {'Application (everything else)': 1,'System (sits between the hardware and the application software e.g. OSs)': 2,
    #                 'Utility (performs specific tasks to keep the computer running e.g. antivirus)':3}
    requirements_volatility = {'Changing': 1,'Fixed': 2}
    requirements_clarity = {'unknown/defined later in the lifecycle': 1,'understandable/early defined': 2}
    dev_time = {'Intensive':1, 'Non-Intensive':2}
    project_size = {'Small':1 , 'Medium':2, 'Large':3}
    team_size = {'Small (1-5)':1, 'Medium (6-15)':2, 'Large (16....)':3}
    prod_complexity = {'Simple':1, 'Complex':2}
    testing_intensity = {'After each cycle (Intensive testing)':1, 'After development is done (Non-intensive testing)':2}


    # df.project_type = [project_type[item] for item in df.project_type]
    df.requirements_volatility = [requirements_volatility[item] for item in df.requirements_volatility]
    df.requirements_clarity = [requirements_clarity[item] for item in df.requirements_clarity]
    df.dev_time = [dev_time[item] for item in df.dev_time]
    df.project_size = [project_size[item] for item in df.project_size]
    df.team_size = [team_size[item] for item in df.team_size]
    df.prod_complexity = [prod_complexity[item] for item in df.prod_complexity]
    df.testing_intensity = [testing_intensity[item] for item in df.testing_intensity]

    return df

In [33]:
def remove_class(df):
    df = df.drop('methodology',1)
    return df

In [34]:
conv = convert_to_vectors(df)
conv

,methodology,requirements_volatility,requirements_clarity,dev_time,project_size,team_size,prod_complexity,testing_intensity,risk_analysis,user_participation,team_expertise,dev_expertise,doc_needed,fund_avail,delivery_speed,task_visualisation
0,Scrum,1,1,1,2,2,1,1,1,3,3,3,1,3,2,1
1,Kanban,1,1,1,2,3,2,1,2,1,3,3,2,3,2,3
2,Scrum,1,1,1,2,2,1,1,1,3,3,3,1,3,2,1
3,Hybrid: Scrum and Waterfall,1,2,1,2,3,2,1,2,3,2,3,3,3,2,1
4,Hybrid: Scrum and Waterfall,1,2,1,2,2,2,1,3,2,3,3,2,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Waterfall,2,2,2,1,1,1,2,3,1,2,2,3,1,1,1
122,Waterfall,2,2,2,1,1,1,2,3,1,2,2,3,1,1,1
123,Waterfall,2,2,2,1,1,1,2,3,1,2,2,3,1,1,1
124,Waterfall,2,2,2,1,1,1,2,3,1,2,2,3,1,1,1


In [35]:
import math
from sklearn import neighbors, datasets
from numpy.random import permutation
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

In [36]:
def euclideanDistance(data1, data2, length):
    distance = 0
    
    #calculating distance
    for x in range(length):
        distance += np.square(data1[x] - data2[x])
    return np.sqrt(distance)

def knn(trainingSet, testInstance, k):
    print(k)
    distances = {}
    sort = {}
    length = testInstance.shape[1]

    for x in range(len(trainingSet)):

        dist = euclideanDistance(testInstance, trainingSet.iloc[x], length)
        
        #getting closest neighbours to the instance
        distances[x] = dist[0]

    #adding the 2nd element in the list to the sort function (neighbour)
    sorted_d = sorted(distances.items(), key=lambda x: x[1])

 
    neighbors = []

    for x in range(k):
        neighbors.append(sorted_d[x][0])

    classVotes = {}

    for x in range(len(neighbors)):
        response = trainingSet.iloc[neighbors[x]][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1

    sortedVotes = sorted(classVotes.items(), key=lambda x: x[1], reverse=True)
 
    return(sortedVotes, neighbors)

In [37]:
# #testing the algorithm 
# testSet = [[1,0,1,0,1,2,1,0,1,0,1,2,2,2,1,0]]
# test = pd.DataFrame(testSet)
# test.shape

In [38]:
cols_test = ['methodology', 'requirements_volatility', 
        'requirements_clarity', 'dev_time', 'project_size', 'team_size', 
        'prod_complexity', 'testing_intensity', 'risk_analysis', 'user_participation',
        'team_expertise', 'dev_expertise', 'doc_needed', 'fund_avail', 'delivery_speed', 'task_visualisation']
# testing the algorithm 
testSet = [['Waterfall', 'Fixed', 'understandable/early defined', 
            'Non-Intensive', 'Small', 'Small (1-5)', 'Simple', 'After development is done (Non-intensive testing)', 'Low', 'High','High', 'High', 
            'Medium','Low','Low', 'Low']]
test = pd.DataFrame(testSet, columns=cols_test)
test


,methodology,requirements_volatility,requirements_clarity,dev_time,project_size,team_size,prod_complexity,testing_intensity,risk_analysis,user_participation,team_expertise,dev_expertise,doc_needed,fund_avail,delivery_speed,task_visualisation
0,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),Low,High,High,High,Medium,Low,Low,Low


In [39]:
testo = convert_to_vectors(test)
testo = remove_class(testo)

/var/folders/b5/rpp5gn7169g6990v79kvffzc0000gp/T/ipykernel_1259/2904615111.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('methodology',1)


In [40]:
testo

,requirements_volatility,requirements_clarity,dev_time,project_size,team_size,prod_complexity,testing_intensity,risk_analysis,user_participation,team_expertise,dev_expertise,doc_needed,fund_avail,delivery_speed,task_visualisation
0,2,2,2,1,1,1,2,1,3,3,3,2,1,1,1


In [41]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(df.iloc[:,1:], df['methodology'])

print(neigh.predict(testo))

['Scrum']


In [45]:
#setting k
k = 500 

result,neigh= knn(conv, test, k)

list1 = []
list2 = []
for i in result:
    list1.append(i[0])
    list2.append(i[1])
for i in list1:
    print(i)

500


KeyError: 0

In [ ]:
df = pd.read_csv('Sample_data.csv')
df.shape


def sdlc_weighted_rating(sdlc, c, m):
    nv = sdlc['tot_count']
    av = sdlc['mean']
    rating = (nv/(nv+m) * av) + (m/(m+nv) * c)
    return rating

In [ ]:
df['Total']= df.iloc[:, 1:5].sum(axis=1)
print (df)

      user_id  Requirements   Risk  Documentation  Stakeholder Involvement  \
0           1              2     9              8                        9   
1           2              3     9              7                        8   
2           3              2     9              7                        9   
3           4              3     8              8                        9   
4           5              2     9              8                        9   
...       ...            ...   ...            ...                      ...   
1051      284              5     3              8                        3   
1052      285              5     3              8                        3   
1053      286              5     2              8                        4   
1054      287              5     3              8                        3   
1055      288              7     2              8                        2   

        PMMname  Total  
0     Waterfall     28  
1     Waterfa

In [ ]:
df['PMMname'].value_counts()
rat = df.groupby('PMMname').agg({'Total': ['mean']})
rat['Total count'] = df['PMMname'].value_counts()
rat['mean'] = df.groupby('PMMname').agg({'Total': ['mean']})
#rat.rename(columns={ rat.columns[0]: "mean" }, inplace = True)
rat= rat.iloc[:, 1:]

rat



,Total count,mean
,,
PMMname,,
Kanban,330,19.000
Scrum,352,17.875
Spiral,220,25.900
Waterfall,154,27.000


In [ ]:
df.set_index('PMMname')
meta = pd.merge(rat,df,on='PMMname')
meta.rename(columns={ meta.columns[2]: "mean" }, inplace = True)
meta.rename(columns={ meta.columns[1]: "tot_count" }, inplace = True)
meta.dtypes


C:\Users\waylo\anaconda3\envs\masters\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


PMMname                     object
tot_count                    int64
mean                       float64
user_id                      int64
Requirements                 int64
Risk                         int64
Documentation                int64
Stakeholder Involvement      int64
dtype: object

In [ ]:
def get_recemmendations(sdlcs, req=0, risk=0, doc =0, stak=0, recomm_count=4, percent=0.7):
    if req != 0:
        sdlcs = sdlcs[sdlcs['Requirements ']>=req]
    if risk != 0:
        sdlcs = sdlcs[sdlcs['Risk']>=risk]   
    if doc != 0:
        sdlcs = sdlcs[sdlcs['Documentation']>=doc]
    if stak != 0:
        sdlcs = sdlcs[sdlcs['Stakeholder Involvement']>=stak]  

    all_sdlcs_avg = sdlcs['mean'].mean()
    min_req_vote = sdlcs['tot_count'].quantile(percent)
    result = sdlcs.copy().loc[sdlcs['tot_count'] >= min_req_vote]

    result['rating'] = result.apply(sdlc_weighted_rating, c=all_sdlcs_avg, m=min_req_vote, axis=1)
    result = result.sort_values('rating',ascending = False)[:recomm_count]

    #result['PMMname'] = result['PMMname'].apply(lambda x: x.PMMname())
    
    return result

In [ ]:
#@title Enter you preferences
#@markdown in the form below, enter your preferences (leave them as default if it doesn't matter)

requirements = 1#@param {type:"integer"}
risks =  9#@param {type:"integer"}
docs =  8#@param {type:"integer"}
stake =7  #@param {type:"integer"}
recommendation_count = 1 #@param {type:"integer"}

In [ ]:
get_recemmendations(meta, req=requirements,risk=risks,
                    doc=docs, stak = stake, recomm_count=1)

,PMMname,tot_count,mean,user_id,Requirements,Risk,Documentation,Stakeholder Involvement,rating
902,Waterfall,154,27.0,1,2,9,8,9,27.0


In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

PMMname = df['PMMname']
characteristics = df['characteristics']
PMM = 'Kanban'


df = pd.DataFrame({'title': PMMname, 'characteristics': characteristics})
df = df[['title', 'characteristics']]

# initialize the new column to hold found keywords
df['Key_words'] = ""

# loop the dataframe created
for index, row in df.iterrows():
    # initialize vectorizer for NLP
    vectorizer = TfidfVectorizer()

    # extracting the words by passing the text
    vectorizer.fit_transform(row)

    # getting the dictionary with key words as keys and their scores as values
    key_words_dict_scores = vectorizer.get_feature_names()

    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores)

# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['Key_words'].astype('str'))

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# gettin the index of the activity that matches the title
idx = df.index[df['title'] == PMM].tolist()[0]

# creating a Series with the similarity scores in descending order
score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)

# getting the indexes of the 5 most similar activities
top_3_indexes = list(score_series.iloc[1:4].index)

# initializing the empty list to hold the recommendations
recommended_activities = list()
for i in top_3_indexes:
    recommended_activities.append((df['title'].loc[i]))

# now remove the original activity to not recommend it back to the user
# fix here
if recommended_activities.__contains__(PMM):
    recommended_activities.remove(PMM)

# append recommendations with comma


KeyError: 'PMMname'

In [ ]:
PMMname = df[['PMMname']]
PMMname

,PMMname
0,Waterfall
1,Waterfall
2,Waterfall
3,Waterfall
4,Waterfall
...,...
955,Kanban
956,Kanban
957,Kanban
958,Kanban
